# AIリテラシ　第９回目
### AIによる予測
___
__スパムメールフィルタ__


データをColaboへドラッグ＆ドロップでアップロードする  
SMSSpamCollection

### データの読み込み
データに列名が無い、タブ区切りの場合

In [ ]:
import pandas as pd
email = pd.read_table('SMSSpamCollection', header=None, names=['type', 'text'])
email

### データの変換
type列をham=>0、spam=>1に変換する

In [ ]:
types = email["type"]
types_01 = types.map({'ham': 0, 'spam': 1})
email.iloc[:, 0] = types_01
email.head()

### ワードクラウドの作成（SPAMメール）
ワードクラウド：頻出語を頻度に比例する大きさで雲のように並べたもの  

注）日本語は単語に分解する（形態素解析）のが難しいので今回は英語のメールを使用します  

参考：https://okumuralab.org/~okumura/python/wordcloud.html  


In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

spam = email[email['type'] == 1]
spam_words = ' '.join(spam['text'])
spam_wc = WordCloud()
spam_wc.generate(spam_words)
plt.imshow(spam_wc)
plt.show()

In [ ]:
for k, v in spam_wc.words_.items():
  if(v >= 0.3):
    print(k, ":", v)

### ワードクラウドの作成（通常メール）

In [ ]:
ham = email[email['type'] == 0]
ham_words = ' '.join(ham['text'])
ham_wc = WordCloud()
ham_wc.generate(ham_words)
plt.imshow(ham_wc)
plt.show()

In [ ]:
for k, v in ham_wc.words_.items():
  if(v >= 0.3):
    print(k, ":", v)

### 大文字を小文字に変換
表記の揺れによる認識の変化が無いようにする  
ex. Apple apple

In [ ]:
email['text'] = email['text'].str.lower()
email

### ストップワードの除去
ストップワード：あまり意味のない単語で対象から除外する単語

In [ ]:
print(list(spam_wc.stopwords)[0:10])
print(list(ham_wc.stopwords)[0:10])

In [ ]:
stop_words_list = list(ham_wc.stopwords)
stop_words_list.extend(["00", "000", "02", "0207"])

### 形態素解析の実行

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vector = CountVectorizer(min_df = 3, stop_words=stop_words_list)
vector.fit(email['text'])
text_vec = vector.transform(email['text'])

dict(list(vector.vocabulary_.items())[0:5])

In [ ]:
# ベクトル化した内容
pd.DataFrame(text_vec.toarray(), columns=vector.get_feature_names())

### スパム分類AIの作成

In [ ]:
from sklearn.naive_bayes import BernoulliNB
model = BernoulliNB()
model.fit(text_vec, email['type'])

### 分類の成功率

In [ ]:
model.score(text_vec, email['type'])

### 正常メールの判定テスト

In [ ]:
test = pd.DataFrame(['I cant pick the phone right now.'])
test_vec = vector.transform(test[0])
model.predict(test_vec)

### スパムメールの判定テスト

In [ ]:
test = pd.DataFrame(['Congratulations ur awarded $500.'])
test_vec = vector.transform(test[0])
model.predict(test_vec)